# Example Use of Dark-MaRK Package

#### Explanation of the package, what it does, and what this tutorial will show, as well as where to find further documentation, examples or help.

Import neccessary packages

In [9]:
import astropy.units as u 
import darkmark
import numpy as np 

Load in the functions from the darkmark package that are needed

In [1]:
from darkmark.boostrap import bootstrap_rate
from darkmark.simload import load_sim_profile 
from darkmark.sampling_transformations import def_samp_coord, calc_real_uE_pE,sample_dm_solcirc 
from darkmark.dm_detector_class import Nibbler 

/Users/gracelawrence/Documents/2021/Dark-MaRK


Define the parameters of the experiment you want to model. Some of these parameters are not called in the example functions below.

In [10]:
v0 = 233 #Average Velocity Speed of Dark Matter in the Halo in km/s.
rho_dm = 0.3 #Dark Matter Density in (Gev/c^2)/cm^3.
M_dm = 60 #Dark Matter Particle Mass in GeV/c^2.
A = 73 #Atomic Mass of the Target Detector Nuclei.
sigma_0 = 1.3e-41*u.cm**2 #Cross-section.
v_max = 600 #Escape velocity of the system in km/s.
period = 365 #The time period to perform the calculates over (typically 1 year) in units of days.
energy_range = np.linspace(0,100,100) #The energy region of interest.
detvar =('Ge') #The detector element to investigate.
dmvar=('CDM') #The Dark Matter cosmology assumed (CDM is currently only option with WDM and SiDM in development).
vdf_var =('SHM') #The velocity model to use, 'SHM' (standard halo model) is currntly the default.

### Define the Nibbler class 

In [ ]:
nib = Nibbler(detvar, dmvar, 'Nucleon', vdf_var, v0,rho_dm, M_dm, sigma_0, v_max,period)                                           

### Define the paths to the simulation data, and to the output folder

In [11]:
file_path = "./Simulations/snapshot_600.1.hdf5"

In [12]:
results_path = "./DarkMark_Example_Outputs/"

In [13]:
n_samples = 8 #8 samples to be taken around the solar circle 

### Load the simulation profile 

In [ ]:
target,s = load_sim_profile(True,file_path,True, True, results_path, cosmology = 'WMAP1') 

Load galaxy realization and profile.

### Generate the co-ordinates at which to samples the dark matter

In [ ]:
sample_coords, theta_ang = def_samp_coord(target, n_samples) 

### Sample the dark matter and generate the velocity disitribution function in both the galactocentric and geocentric reference frames, for each of the samples.

In [ ]:
vel_galacto, vel_geo = sample_dm_solcirc(nib,sample_coords, target, theta_ang, 'milkyway',results_path, False)   

Generate an example VDF plot comparing the galactocentric and geocentric distributions with a MB fit.

### Integrate over the velocity distributions to generate spectral rate functions.

In [ ]:
bootstrap_rate(nib, n_samples, results_path, False) 

Generate a spectral rate function plot